In [1]:
# ===== SALES ANALYSIS PROJECT =====

# Part II: SQL Queries for Analysis

In [2]:
# Import libraries

import pandas as pd
import os
from sqlalchemy import create_engine
from sqlalchemy import text

# Fetch environment variables
username = os.getenv('MYSQL_USER')
password = os.getenv('MYSQL_PASSWORD')
host = os.getenv('MYSQL_HOST')
database = os.getenv('MYSQL_DB')

# Create sqlalchemy engine
engine = create_engine(f'mysql+pymysql://{username}:{password}@{host}/{database}')

# Formats all floats to two decimal places
pd.options.display.float_format = '{:,.2f}'.format 

In [3]:
# Test Query: Retrieve column names and sample rows

query = "SELECT * FROM sales_data LIMIT 5;"
sample_data = pd.read_sql(query, engine)
sample_data.head()

,row_id,order_id,product,quantity_ordered,price_each,order_date,street,city,state,zip,total_price
0,1,"176,558.00",USB-C Charging Cable,2.00,11.95,2019-04-19 08:46:00,917 1st St,Dallas,TX,75001,23.90
1,2,"176,559.00",Bose SoundSport Headphones,1.00,99.99,2019-04-07 22:30:00,682 Chestnut St,Boston,MA,02215,99.99
2,3,"176,560.00",Google Phone,1.00,600.00,2019-04-12 14:38:00,669 Spruce St,Los Angeles,CA,90001,600.00
3,4,"176,560.00",Wired Headphones,1.00,11.99,2019-04-12 14:38:00,669 Spruce St,Los Angeles,CA,90001,11.99
4,5,"176,561.00",Wired Headphones,1.00,11.99,2019-04-30 09:27:00,333 8th St,Los Angeles,CA,90001,11.99


In [4]:
# I had to use .text() to wrap the queries bc .read_sql was not interpreting the raw SQL string 
# Comments for SQL queries are included in the queries themselves, denoted by "--" at the beginning of each comment

sql_sales_by_month = text("""
-- Total Sales by Month
SELECT 
    DATE_FORMAT(order_date, '%m') AS month,
    SUM(total_price) AS monthly_sales
FROM
    sales_data
GROUP BY month
ORDER BY monthly_sales DESC;
""")

total_sales_by_month = pd.read_sql(sql_sales_by_month, engine)
total_sales_by_month.head(10)

,month,monthly_sales
0,12,"4,608,295.70"
1,10,"3,734,777.86"
2,04,"3,389,217.98"
3,11,"3,197,875.05"
4,05,"3,150,616.23"
5,03,"2,804,973.35"
6,07,"2,646,461.32"
7,06,"2,576,280.15"
8,08,"2,241,083.37"
9,02,"2,200,078.08"


In [5]:
sql_sales_by_dayofweek = text( """
-- Total Sales by Day of the Week
SELECT 
    DAYNAME(order_date) AS day_of_week,
    SUM(total_price) AS daily_sales
FROM
    sales_data
GROUP BY day_of_week
ORDER BY daily_sales DESC;
""")

total_sales_by_dayofweek = pd.read_sql(sql_sales_by_dayofweek, engine)
total_sales_by_dayofweek.head(10)

,day_of_week,daily_sales
0,Tuesday,"5,086,275.40"
1,Wednesday,"4,986,823.36"
2,Sunday,"4,927,249.40"
3,Saturday,"4,900,195.58"
4,Monday,"4,877,588.21"
5,Friday,"4,853,642.26"
6,Thursday,"4,833,763.73"


In [6]:
sql_sales_by_day = text( """
-- Total Sales by Day
SELECT 
    DATE(order_date) AS day, SUM(total_price) AS daily_sales
FROM
    sales_data
GROUP BY day
ORDER BY daily_sales DESC
LIMIT 10;
""")

total_sales_by_day = pd.read_sql(sql_sales_by_day, engine)
total_sales_by_day.head(10)

,day,daily_sales
0,2019-12-04,"166,577.69"
1,2019-12-16,"162,970.61"
2,2019-12-10,"162,820.75"
3,2019-12-20,"160,178.31"
4,2019-12-17,"157,357.04"
5,2019-12-29,"156,009.67"
6,2019-12-05,"155,079.30"
7,2019-12-21,"154,656.88"
8,2019-12-18,"154,598.77"
9,2019-12-24,"152,888.82"


In [7]:
sql_sales_by_hour = text("""
-- Total Sales by Hour
SELECT 
    HOUR(order_date) AS hour, SUM(total_price) AS hourly_sales
FROM
    sales_data
GROUP BY hour
ORDER BY hourly_sales DESC;
""")

total_sales_by_hour = pd.read_sql(sql_sales_by_hour, engine)
total_sales_by_hour.head(12)

,hour,hourly_sales
0,19,"2,411,971.14"
1,12,"2,314,359.85"
2,11,"2,296,619.84"
3,20,"2,280,784.36"
4,18,"2,218,374.01"
5,13,"2,152,369.98"
6,17,"2,126,553.97"
7,14,"2,082,513.70"
8,21,"2,040,790.48"
9,10,"1,942,988.08"


In [8]:
sql_repeat_v_onetime = text("""
-- Number of Repeat Customers vs One-time Customers 
SELECT 
    CASE
        WHEN order_count = 1 THEN 'One-time Customer'
        ELSE 'Repeat Customer'
    END AS customer_type,
    COUNT(*) AS customer_count
FROM
    (SELECT 
        order_id, COUNT(*) AS order_count
    FROM
        sales_data
    GROUP BY order_id) customer_orders
GROUP BY customer_type;
""")

number_repeat_v_onetime = pd.read_sql(sql_repeat_v_onetime, engine)
number_repeat_v_onetime.head()

,customer_type,customer_count
0,One-time Customer,171558
1,Repeat Customer,6879


In [9]:
sql_top5_quantity_repeat_v_onetime = text("""
-- Top 5 Products by Repeat Customer vs One-time Customer
WITH customer_orders AS (
    -- Identify One-time vs Repeat Customers using order_id
    SELECT order_id, COUNT(*) AS order_count
    FROM sales_data
    GROUP BY order_id
),
customer_classification AS (
    -- Classify Orders into One-time or Repeat Customers
    SELECT 
        order_id,
        CASE 
            WHEN order_count = 1 THEN 'One-time Customer'
            ELSE 'Repeat Customer'
        END AS customer_type
    FROM customer_orders
),
customer_product_sales AS (
    -- Join with sales_data to get Product Purchases
    SELECT 
        c.customer_type,
        s.product AS product_name,
        SUM(s.quantity_ordered) AS total_quantity
    FROM sales_data s
    JOIN customer_classification c ON s.order_id = c.order_id
    GROUP BY c.customer_type, s.product
)
-- Top 5 Products for One-time or Repeat Customers
SELECT * FROM (
    SELECT 
        customer_type,
        product_name,
        total_quantity,
        RANK() OVER (PARTITION BY customer_type ORDER BY total_quantity DESC) AS rank_order
    FROM customer_product_sales
) ranked
WHERE rank_order <= 5;
""")

top5_quantity_repeat_v_onetime = pd.read_sql(sql_top5_quantity_repeat_v_onetime, engine)
top5_quantity_repeat_v_onetime.head(10)

,customer_type,product_name,total_quantity,rank_order
0,One-time Customer,AAA Batteries (4-pack),"29,797.00",1
1,One-time Customer,AA Batteries (4-pack),"26,642.00",2
2,One-time Customer,USB-C Charging Cable,"21,725.00",3
3,One-time Customer,Lightning Charging Cable,"21,313.00",4
4,One-time Customer,Wired Headphones,"18,764.00",5
5,Repeat Customer,USB-C Charging Cable,"2,206.00",1
6,Repeat Customer,iPhone,"1,864.00",2
7,Repeat Customer,Lightning Charging Cable,"1,856.00",3
8,Repeat Customer,Wired Headphones,"1,760.00",4
9,Repeat Customer,Google Phone,"1,636.00",5


In [10]:
sql_top5_sales_repeat_v_onetime = text("""
-- Top 5 Products by Sales for Repeat Customers and One-time Customers
WITH customer_orders AS (
    -- Identify One-time vs Repeat Customers using order_id
    SELECT order_id, COUNT(*) AS order_count
    FROM sales_data
    GROUP BY order_id
),
customer_classification AS (
    -- Classify Orders into One-time or Repeat Customers
    SELECT 
        order_id,
        CASE 
            WHEN order_count = 1 THEN 'One-time Customer'
            ELSE 'Repeat Customer'
        END AS customer_type
    FROM customer_orders
),
customer_product_sales AS (
    -- Join with sales_data to get Product Purchases by Total Revenue
    SELECT 
        c.customer_type,
        s.product AS product_name,
        SUM(s.total_price) AS total_sales  
    FROM sales_data s
    JOIN customer_classification c ON s.order_id = c.order_id
    GROUP BY c.customer_type, s.product
)
-- Get the Top 5 Products for Each Customer Type (One-time vs. Repeat)
SELECT * FROM (
    SELECT 
        customer_type,
        product_name,
        total_sales,  
        RANK() OVER (PARTITION BY customer_type ORDER BY total_sales DESC) AS rank_order
    FROM customer_product_sales
) ranked
WHERE rank_order <= 5;
""")

top5_sales_repeat_v_onetime = pd.read_sql(sql_top5_sales_repeat_v_onetime, engine)
top5_sales_repeat_v_onetime.head(10)

,customer_type,product_name,total_sales,rank_order
0,One-time Customer,Macbook Pro Laptop,"7,707,800.00",1
1,One-time Customer,ThinkPad Laptop,"3,955,960.44",2
2,One-time Customer,iPhone,"3,488,100.00",3
3,One-time Customer,27in 4K Gaming Monitor,"2,339,160.02",4
4,One-time Customer,Google Phone,"2,335,800.00",5
5,Repeat Customer,iPhone,"1,304,800.00",1
6,Repeat Customer,Google Phone,"981,600.00",2
7,Repeat Customer,Macbook Pro Laptop,"324,700.00",3
8,Repeat Customer,Vareebadd Phone,"240,800.00",4
9,Repeat Customer,ThinkPad Laptop,"171,998.28",5


In [11]:
sql_ltv_repeat_v_onetime = text("""
-- Lifetime Value of Repeat and One-Time Customers and their Average Order Value
WITH customer_orders AS (
    -- Count how many times each order_id appears and sum their total_price
    SELECT order_id, SUM(total_price) AS total_spent, COUNT(order_id) AS order_count
    FROM sales_data
    GROUP BY order_id
),
customer_classification AS (
    -- Label Orders as One-time or Repeat Customers
    SELECT 
        order_id,
        total_spent,
        CASE 
            WHEN order_count = 1 THEN 'One-time Customer'
            ELSE 'Repeat Customer'
        END AS customer_type
    FROM customer_orders
)
-- Calculate Lifetime Value by Customer Type
SELECT 
    customer_type,
    COUNT(order_id) AS total_customers,
    SUM(total_spent) AS total_revenue,
    AVG(total_spent) AS avg_lifetime_value
FROM customer_classification
GROUP BY customer_type;
""")

ltv_repeat_v_onetime = pd.read_sql(sql_ltv_repeat_v_onetime, engine)
ltv_repeat_v_onetime.head()

,customer_type,total_customers,total_revenue,avg_lifetime_value
0,One-time Customer,171558,"30,818,273.75",179.64
1,Repeat Customer,6879,"3,647,264.19",530.20


In [12]:
sql_products_sold_together = text("""
-- Products Frequently Purchased Together
SELECT 
    LEAST(a.product, b.product) AS product_1, 
    GREATEST(a.product, b.product) AS product_2, 
    COUNT(*) AS purchase_count
FROM sales_data a
JOIN sales_data b 
    ON a.order_id = b.order_id 
    AND a.product <> b.product
GROUP BY product_1, product_2
ORDER BY purchase_count DESC
LIMIT 20;
""")

products_most_sold_together = pd.read_sql(sql_products_sold_together, engine)
products_most_sold_together.head(20)

,product_1,product_2,purchase_count
0,iPhone,Lightning Charging Cable,2022
1,Google Phone,USB-C Charging Cable,1994
2,iPhone,Wired Headphones,924
3,Google Phone,Wired Headphones,844
4,Apple Airpods Headphones,iPhone,746
5,USB-C Charging Cable,Vareebadd Phone,736
6,Bose SoundSport Headphones,Google Phone,456
7,USB-C Charging Cable,Wired Headphones,406
8,Vareebadd Phone,Wired Headphones,298
9,Lightning Charging Cable,Wired Headphones,258


In [13]:
sql_missed_product_bundles = text("""
-- Commonly Bundled Products That Were Purchased Separately
-- Step 1: Top 10 Frequently Purchased Together Product Pairs
WITH top_product_pairs AS (
    SELECT 
        a.product AS product_1, 
        b.product AS product_2, 
        COUNT(*) AS purchase_count
    FROM sales_data a
    JOIN sales_data b 
        ON a.order_id = b.order_id 
        AND a.product <> b.product  -- Ensures it's not the same product
    GROUP BY product_1, product_2
    ORDER BY purchase_count DESC
    LIMIT 20
)

-- Step 2: Count How Often Each Product Was Purchased Alone
SELECT 
    p.product, 
    COUNT(*) AS purchase_alone_count
FROM sales_data p
WHERE p.product IN (
    -- Get all products from the top 10 pairs
    SELECT product_1 FROM top_product_pairs
    UNION
    SELECT product_2 FROM top_product_pairs
)
AND p.order_id NOT IN (
    -- Exclude orders that contain both products in a top 10 pair
    SELECT DISTINCT a.order_id 
    FROM sales_data a
    JOIN sales_data b 
        ON a.order_id = b.order_id 
        AND a.product <> b.product
    JOIN top_product_pairs tp 
        ON (a.product = tp.product_1 AND b.product = tp.product_2)
)
GROUP BY p.product
ORDER BY purchase_alone_count DESC;
""")

missed_product_bundles = pd.read_sql(sql_missed_product_bundles, engine)
missed_product_bundles.head(20)

,product,purchase_alone_count
0,Lightning Charging Cable,20531
1,USB-C Charging Cable,20410
2,Wired Headphones,17671
3,Apple Airpods Headphones,15147
4,Bose SoundSport Headphones,13050
5,iPhone,5126
6,Google Phone,4017
7,Vareebadd Phone,1580


In [14]:
missed_product_bundles['purchase_alone_count'].sum()

97532

In [15]:
sql_zip_by_sales = text("""
-- Zip Codes by Total Sales (There are only Ten Distinct Zip Codes in the Dataset)
SELECT 
    zip, SUM(total_price) AS total_sales
FROM
    sales_data
GROUP BY zip
ORDER BY total_sales DESC;
""")

zip_by_sales = pd.read_sql(sql_zip_by_sales, engine)
zip_by_sales.head(20)

,zip,total_sales
0,94016,"8,254,743.55"
1,90001,"5,448,304.28"
2,10001,"4,661,867.14"
3,02215,"3,658,627.65"
4,30301,"2,794,199.07"
5,75001,"2,765,373.96"
6,98101,"2,745,046.02"
7,97035,"1,870,010.56"
8,73301,"1,818,044.33"
9,04101,"449,321.38"


In [16]:
sql_top5_quantity_by_top5_zip = text("""
-- Top 5 Most Sold Products for Each of the Top 5 Zip Codes by Total Sales
-- Step 1: Get the Top 5 Zip Codes by Total Sales
WITH top_zipcodes AS (
    SELECT 
        zip, 
        SUM(total_price) AS total_sales
    FROM sales_data
    GROUP BY zip
    ORDER BY total_sales DESC
    LIMIT 5  -- Fix: Changed from 10 to 5
)
-- Step 2: Get the Top 5 Products Sold in Each of These Zip Codes
SELECT s.zip, s.product, s.total_quantity_sold, s.rank_order
FROM (
    SELECT 
        s.zip, 
        s.product, 
        SUM(s.quantity_ordered) AS total_quantity_sold,
        RANK() OVER (PARTITION BY s.zip ORDER BY SUM(s.quantity_ordered) DESC) AS rank_order
    FROM sales_data s
    JOIN top_zipcodes tz ON s.zip = tz.zip
    GROUP BY s.zip, s.product
) s
WHERE s.rank_order <= 5  -- Fix: Changed from <= 1 to <= 5
ORDER BY s.zip, s.rank_order;
""")

top5_quantity_by_top5_zip = pd.read_sql(sql_top5_quantity_by_top5_zip, engine)
top5_quantity_by_top5_zip.head(25)

,zip,product,total_quantity_sold,rank_order
0,02215,AAA Batteries (4-pack),"3,458.00",1
1,02215,AA Batteries (4-pack),"3,011.00",2
2,02215,USB-C Charging Cable,"2,555.00",3
3,02215,Lightning Charging Cable,"2,482.00",4
4,02215,Wired Headphones,"2,222.00",5
5,10001,AAA Batteries (4-pack),"4,119.00",1
6,10001,AA Batteries (4-pack),"3,629.00",2
7,10001,USB-C Charging Cable,"3,263.00",3
8,10001,Lightning Charging Cable,"3,039.00",4
9,10001,Wired Headphones,"2,702.00",5


In [17]:
sql_top5_product_by_totalsales_by_top5_zip = text("""
-- Top 5 Products by Total Sales for Each of the Top 5 Zip Codes by Total Sales
-- Step 1: Get the Top 5 Zip Codes by Total Sales
WITH top_zipcodes AS (
    SELECT 
        zip, 
        SUM(total_price) AS total_sales
    FROM sales_data
    GROUP BY zip
    ORDER BY total_sales DESC
    LIMIT 5  -- Fix: Changed from 10 to 5
)
-- Step 2: Get the Top 5 Products Sold in Each of These Zip Codes by Total Revenue
SELECT s.zip, s.product, s.total_sales, s.rank_order
FROM (
    SELECT 
        s.zip, 
        s.product, 
        SUM(s.total_price) AS total_sales,
        RANK() OVER (PARTITION BY s.zip ORDER BY SUM(s.total_price) DESC) AS rank_order
    FROM sales_data s
    JOIN top_zipcodes tz ON s.zip = tz.zip
    GROUP BY s.zip, s.product
) s
WHERE s.rank_order <= 5  -- Fix: Changed from <= 1 to <= 5
ORDER BY s.zip, s.rank_order;

""")

top5_product_by_totalsales_by_top5_zip = pd.read_sql(sql_top5_product_by_totalsales_by_top5_zip, engine)
top5_product_by_totalsales_by_top5_zip.head(25)

,zip,product,total_sales,rank_order
0,02215,Macbook Pro Laptop,"814,300.00",1
1,02215,iPhone,"526,400.00",2
2,02215,ThinkPad Laptop,"446,995.53",3
3,02215,Google Phone,"355,200.00",4
4,02215,27in 4K Gaming Monitor,"263,243.25",5
5,10001,Macbook Pro Laptop,"1,116,900.00",1
6,10001,iPhone,"616,700.00",2
7,10001,ThinkPad Laptop,"559,994.40",3
8,10001,Google Phone,"454,200.00",4
9,10001,34in Ultrawide Monitor,"329,451.33",5


In [18]:
# Dispose of the engine

engine.dispose()  
print("Engine disposed")

Engine disposed


In [19]:
# Part II: SQL Queries for Analysis is complete!

# Time to distill these result sets into insights that drive business!